In [1]:
!pip install pulp

import pulp
import numpy as np
from math import radians, cos, sin, asin, sqrt, exp
import pandas as pd

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points on the earth.
    Uses the Haversine formula to compute the shortest distance over the earth's surface.
    """
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 3440  # Radius of Earth in nautical miles
    return c * r

def create_distance_matrix(ports_df):
    """Create matrix of distances between all ports"""
    n = len(ports_df)
    distances = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            if i != j:
                distances[i, j] = haversine_distance(
                    ports_df.iloc[i]['latitude'],
                    ports_df.iloc[i]['longitude'],
                    ports_df.iloc[j]['latitude'],
                    ports_df.iloc[j]['longitude']
                )
    return distances

def calculate_logistical_cost(distance):
    """
    Calculate logistical cost based on distance using the provided exponential formula.
    """
    return 1696.3 * exp(5e-5 * distance)

def calculate_environmental_cost(distance):
    """
    Calculate environmental cost based on distance using the provided power formula.
    """
    return 0.0864 * (distance ** 0.9708)

def validate_vrp_data(ports_df, vessels_df):
    """
    Validate input data for VRP feasibility.
    Checks various conditions that must be met for a feasible solution.
    """
    # Verify depot existence and properties
    if ports_df.iloc[0]['demand'] != 0:
        return False, "El primer puerto debe ser el depósito con demanda cero"

    # Check total capacity vs total demand
    total_demand = ports_df['demand'].sum()
    total_capacity = vessels_df['capacity'].sum()
    if total_demand > total_capacity:
        return False, f"La demanda total ({total_demand} TEUs) excede la capacidad total ({total_capacity} TEUs)"

    # Check individual port demands vs vessel capacities
    max_port_demand = ports_df['demand'].max()
    max_vessel_capacity = vessels_df['capacity'].max()
    if max_port_demand > max_vessel_capacity:
        return False, f"La demanda máxima de un puerto ({max_port_demand} TEUs) excede la capacidad máxima de un buque ({max_vessel_capacity} TEUs)"

    # Check coordinate validity
    if not all((-90 <= lat <= 90) for lat in ports_df['latitude']):
        return False, "Latitudes deben estar entre -90 y 90 grados"
    if not all((-180 <= lon <= 180) for lon in ports_df['longitude']):
        return False, "Longitudes deben estar entre -180 y 180 grados"

    return True, "Datos válidos para VRP"

def solve_maritime_vrp(ports_df, vessels_df, alpha=0.5):
    """
    Solve the Maritime Vehicle Routing Problem with MTZ formulation.
    This implementation uses a depot-based formulation with Miller-Tucker-Zemlin (MTZ)
    subtour elimination constraints. Each vessel must start and end its route at the depot.
    """
    # Validate input data
    is_valid, message = validate_vrp_data(ports_df, vessels_df)
    if not is_valid:
        print(message)
        return None

    # Ensure alpha is between 0 and 1
    if alpha < 0 or alpha > 1:
        print("Error: alpha debe estar entre 0 y 1.")
        return None

    # Calculate beta as 1 - alpha
    beta = 1 - alpha

    # Create distance matrix
    distances = create_distance_matrix(ports_df)

    # Create optimization model
    prob = pulp.LpProblem("Maritime_VRP", pulp.LpMinimize)

    # Sets
    ports = range(len(ports_df))  # Port 0 is the depot
    vessels = range(len(vessels_df))
    clients = range(1, len(ports_df))  # Exclude depot

    # Decision Variables
    # x[i,j,v] = 1 if vessel v travels from port i to j
    x = pulp.LpVariable.dicts("route",
                              ((i, j, v) for i in ports
                               for j in ports
                               for v in vessels
                               if i != j),
                              cat='Binary')

    # u[i,v] = Position of port i in route of vessel v (MTZ variables)
    u = pulp.LpVariable.dicts("position",
                              ((i, v) for i in ports for v in vessels),
                              lowBound=0,
                              upBound=len(ports) - 1,
                              cat='Integer')

    # Objective Function
    prob += pulp.lpSum(
        alpha * calculate_logistical_cost(distances[i][j]) * x[i, j, v] +
        beta * calculate_environmental_cost(distances[i][j]) * ports_df.iloc[j]['demand'] * x[i, j, v]
        for i in ports
        for j in ports
        for v in vessels
        if i != j
    )

    # Constraints
    # 1. Each client must be visited exactly once
    for j in clients:
        prob += pulp.lpSum(x[i, j, v]
                           for i in ports
                           for v in vessels
                           if i != j) == 1

    # 2. Flow conservation for each vessel
    for v in vessels:
        # Vessel must leave depot
        prob += pulp.lpSum(x[0, j, v] for j in clients) <= 1

        # Vessel must return to depot
        prob += pulp.lpSum(x[i, 0, v] for i in clients) <= 1

        # Flow conservation at client nodes
        for h in clients:
            prob += pulp.lpSum(x[i, h, v] for i in ports if i != h) == \
                    pulp.lpSum(x[h, j, v] for j in ports if j != h)

    # 3. Capacity constraints for each vessel
    for v in vessels:
        prob += pulp.lpSum(ports_df.iloc[j]['demand'] * x[i, j, v]
                           for i in ports
                           for j in clients
                           if i != j) <= vessels_df.iloc[v]['capacity']

    # 4. MTZ subtour elimination constraints
    M = len(ports)  # Big M constant
    for v in vessels:
        for i in clients:
            for j in clients:
                if i != j:
                    prob += u[i, v] - u[j, v] + M * x[i, j, v] <= M - 1

    # Solve the problem
    print("\nResolviendo el modelo de optimización...")
    solver = pulp.PULP_CBC_CMD(msg=False, timeLimit=120)
    prob.solve(solver)

    print(f"Estado de la solución: {pulp.LpStatus[prob.status]}")

    if pulp.LpStatus[prob.status] == 'Optimal':
        # Calculate total logistical and environmental costs (weighted by alpha and beta)
        total_logistical_cost = pulp.value(
            pulp.lpSum(
                alpha * calculate_logistical_cost(distances[i][j]) * x[i, j, v]
                for i in ports
                for j in ports
                for v in vessels
                if i != j
            )
        )

        total_environmental_cost = pulp.value(
            pulp.lpSum(
                beta * calculate_environmental_cost(distances[i][j]) * ports_df.iloc[j]['demand'] * x[i, j, v]
                for i in ports
                for j in ports
                for v in vessels
                if i != j
            )
        )

        solution = {
            'alpha': alpha,
            'beta': beta,
            'total_cost': pulp.value(prob.objective),
            'total_logistical_cost': total_logistical_cost,
            'total_environmental_cost': total_environmental_cost,
            'routes': []
        }

        # Extract routes for each vessel
        for v in vessels:
            route = []
            current = 0  # Start from depot
            visited = {0}

            while True:
                next_port = None
                for j in ports:
                    if j not in visited and j != current:
                        if pulp.value(x[current, j, v]) > 0.5:
                            next_port = j
                            break

                if next_port is None:
                    if len(route) > 0:  # If route exists, close it back to depot
                        route.append((ports_df.iloc[current]['port_id'], 'DEPOT'))
                    break

                route.append((ports_df.iloc[current]['port_id'], ports_df.iloc[next_port]['port_id']))
                visited.add(next_port)
                current = next_port

            if route:
                vessel_solution = {
                    'vessel_id': vessels_df.iloc[v]['vessel_id'],
                    'route': route,
                    'loads': [],
                    'logistical_costs': [],
                    'environmental_costs': []
                }

                # Calculate loads, logistical and environmental costs for each leg
                served_ports = set()
                for port_from, port_to in route:
                    if port_to != 'DEPOT':
                        to_idx = ports_df[ports_df['port_id'] == port_to].index[0]
                        served_ports.add(to_idx)
                        current_load = sum(ports_df.iloc[p]['demand'] for p in served_ports)
                        from_idx = ports_df[ports_df['port_id'] == port_from].index[0]
                        distance = distances[from_idx][to_idx]
                        logistical_cost = calculate_logistical_cost(distance)
                        environmental_cost = calculate_environmental_cost(distance) * ports_df.iloc[to_idx]['demand']
                        vessel_solution['loads'].append({
                            'from': port_from,
                            'to': port_to,
                            'containers': current_load
                        })
                        vessel_solution['logistical_costs'].append(logistical_cost)
                        vessel_solution['environmental_costs'].append(environmental_cost)

                solution['routes'].append(vessel_solution)

        return solution
    else:
        print("No se encontró una solución óptima. Verifique las restricciones del problema.")
        return None

def sensitivity_analysis(ports_df, vessels_df, alpha_values):
    """
    Perform sensitivity analysis by varying alpha values.
    Returns a list of results for each alpha value.
    """
    results = []

    for alpha in alpha_values:
        print(f"\nEjecutando modelo con alpha={alpha}...")
        solution = solve_maritime_vrp(ports_df, vessels_df, alpha=alpha)

        if solution:
            results.append(solution)
        else:
            print("No se encontró una solución factible para estos parámetros.")

    return results

def main():
    """
    Example usage of the Maritime VRP solver with sensitivity analysis.
    """
    # Data
    ports_data = {
        'port_id': ['Depot', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9'],
        'name': ['DEPOT', 'Baltimore', 'Charleston', 'Miami', 'New_Orleans', 'New_York', 'Norfolk', 'Philadelphia', 'Port_Everglades', 'Mariel'],
        'latitude': [39.29038, 39.29038, 32.77657, 25.7741667, 29.95465, 40.71427, 36.84681, 39.95233, 25.8576, 22.9919],
        'longitude': [-76.61219, -76.61219, -79.93092, -80.171111, -90.07507, -74.00597, -76.28522, -75.16379, -81.3866, -82.7667],
        'demand': [0, 1205, 1360, 602, 711, 7412, 1753, 822, 555, 171],  # Depot has no demand
        'max_vessel_size': [23000, 14000, 14000, 14000, 10000, 18000, 18000, 10000, 10000, 10000]
    }

    vessels_data = {
        'vessel_id': ['Feeder', 'Feedermax', 'Handysize', 'Sub-PanaMax', 'Panamax', 'Post-PanaMax', 'Neo-Panamax', 'Ultra Large Container Ship', 'Mega Container Ship'],
        'capacity': [1000, 2000, 3000, 5000, 7000, 10000, 14000, 20000, 23000],  # Capacidades mayores que permiten múltiples puertos
        'speed': [19, 19, 19, 20, 18, 18, 20, 20, 20]
    }

    ports_df = pd.DataFrame(ports_data)
    vessels_df = pd.DataFrame(vessels_data)

    print("\nResolviendo el problema VRP marítimo...")
    print(f"Demanda total de contenedores: {ports_df['demand'].sum()} TEUs")
    print(f"Capacidad total de la flota: {vessels_df['capacity'].sum()} TEUs")

    # Validate input data
    is_valid, message = validate_vrp_data(ports_df, vessels_df)
    if not is_valid:
        print(f"\nError de validación: {message}")
        return

    # Define alpha values for sensitivity analysis
    alpha_values = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]  # Valores de alpha a probar

    # Perform sensitivity analysis
    results = sensitivity_analysis(ports_df, vessels_df, alpha_values)

    # Display sensitivity analysis results
    print("\nResultados del análisis de sensibilidad:")
    for result in results:
        print(f"\nAlpha: {result['alpha']}, Beta: {result['beta']}")
        print(f"Costo total: ${result['total_cost']:,.2f}")
        print(f"Costo logístico total: ${result['total_logistical_cost']:,.2f}")
        print(f"Costo ambiental total: ${result['total_environmental_cost']:,.2f}")
        print("Rutas por buque:")
        for vessel_route in result['routes']:
            print(f"\nBuque {vessel_route['vessel_id']}:")
            for i, (port_from, port_to) in enumerate(vessel_route['route']):
                load = vessel_route['loads'][i]['containers'] if i < len(vessel_route['loads']) else 0
                print(f"  {port_from} -> {port_to} (Carga: {load} TEUs)")

if __name__ == "__main__":
    main()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 31.5 MB/s eta 0:00:00

Resolviendo el problema VRP marítimo...
Demanda total de contenedores: 14591 TEUs
Capacidad total de la flota: 85000 TEUs

Ejecutando modelo con alpha=0.0...

Resolviendo el modelo de optimización...
Estado de la solución: Optimal

Ejecutando modelo con alpha=0.2...

Resolviendo el modelo de optimización...
Estado de la solución: Optimal

Ejecutando modelo con alpha=0.4...

Resolviendo el modelo de optimización...
Estado de la solución: Optimal

Ejecutando modelo con alpha=0.6...

Resolviendo el modelo de optimización...
Estado de la solución: Optimal

Ejecutando modelo con alpha=0.8...

Resolviendo el modelo de optimización...
Estado de la solución: Optimal

Ejecutando modelo con alpha=1.0...

Resolviendo el modelo de optimización...


KeyboardInterrupt: 